## Authoritarian Framework

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessage, _convert_to_message
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [16]:
unique_id = "Authoritarian Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [17]:
# from langsmith import Client

# client = Client()

In [18]:
class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])


class AdvisorAgent:
    """
    Advisor Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, llm = ChatOpenAI(temperature=1.0, model="gpt-4o")):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""
        self.llm = llm

    def review_prompt(self, state: Sequence[BaseMessage], criteria: str):
        """
        Generates a review of the prompt.
        """
        prompt_text = f"""Your task is to review the conversation above and think outside the box to provide feedback on the prompt.
Offer creative recommendations on how to improve it in light of your core principles.
Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement.

The success criteria for the updated prompt are as follows:
{criteria}
You must use this information to inform your feedback.

Below are strict guidelines that MUST be followed if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces.
- ALWAYS treat placeholders as the actual content.
Check that that the prompt adheres to these guidelines.

Your reviewal process should be as follows:
1. Read the conversation carefully as an expert {self.position}.
2. Explcitly go through each success criteria and ensure the prompt meets them. If not, mention the criteria that was not met in your feedback.
3. Explicitly go through each guideline and ensure the changes adhere to them. If not, mention the guideline that was not followed in your feedback.
4. Explicitly list the creative recommendations you have for improving the most critical aspects of the prompt.
5. Submit your feedback.
"""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", prompt_text),
                MessagesPlaceholder(variable_name="messages"),
                # MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})       
        return {"messages": [HumanMessage(content=result.content, name=self.position)]}
        
    def approval(self, state: Sequence[BaseMessage], criteria: str):
        """
        Agent to approve or reject the prompt.
        """
        function_def = {
        "name": "approval",
        "description": "Submit approval decision for the prompt.",
        "parameters": {
            "type": "object",
            "properties": {
                "advisor": {"type": "string", "enum": [self.position]},
                "decision": {"type": "string", "enum": ["True", "False"]},
            },
            "required": ["decision", "advisor_position"],
        },
        }
        prompt_text = f"""Your task is to review the conversation above and decide if the prompt is optimal in light of your core principles and the success criteria.

The success criteria for the prompt are as follows:
{criteria}
You must use this information to inform your decision.

If you think the prompt sufficiently meets the success criteria, return True. 
If you think the prompt needs improvements in light of your core principles to better meet the success criteria, return False.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {self.position}.
2. Determine whether the prompt needs improvement or meets the success criteria.
3. Submit your decision.
"""
        messages = [
            ("system", self.system_message),
            MessagesPlaceholder(variable_name="messages"),
            ("system", prompt_text),
        ]

        prompt = ChatPromptTemplate.from_messages(messages)

        chain = (
            prompt
            | self.llm.bind_functions(functions=[function_def], function_call="approval")
            | JsonOutputFunctionsParser()
        )
        result = chain.invoke({"messages": state})
        return result


In [19]:
class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates with advisor agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str = None, advisors: List[AdvisorAgent] = None, llm = ChatOpenAI(temperature=1.0, model="gpt-4o")):
        self.base_prompt = base_prompt
        self.criteria = criteria
        self.system_message = f"""You are an experienced: Head AI Prompt Engineer. Your core principles are:
- Always pay attention to detail in prompts
- Always be critical of prompts
- Always make informed decisions with regards to prompts"""
        self.llm = llm
        self.advisors = advisors
        self.iterations = 0
    
    def run_approval(self, state: Sequence[BaseMessage]) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(advisor.approval, state, self.criteria) for advisor in self.advisors]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next advisor or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state)
        print("Approval results:", approval_results)
        # Extract decisions from the results
        approval_results = [result["decision"] == "True" for result in approval_results]
        if all(approval_results) or self.iterations > 10:
            return "FINISH"
        else:
            # Only ask for advise from advisors that disapproved the prompt
            disapproved_advisors = [advisor for advisor, result in zip(self.advisors, approval_results) if not result]
            disapproved_advisors_details = [f"{advisor.position}:\n{advisor.core_principles}\n\n" for advisor in self.advisors]
            options = [advisor.position for advisor in disapproved_advisors]
            # shuffle the options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + positions
            function_def = {
                "name": "route",
                "description": "Select the next role.",
                "parameters": {
                    "title": "routeSchema",
                    "type": "object",
                    "properties": {
                        "next": {
                            "title": "Next",
                            "anyOf": [
                                {"enum": options},
                            ],
                        }
                    },
                    "required": ["next"],
                },
            }
            prompt_text = f"""Your task is to review the conversation above and decide the next advisor to provide feedback.

The success criteria for the updated prompt are as follows:
{self.criteria}
You must use this information to inform your decision.

The details of all advisors are their core principles are as follows: 
{disapproved_advisors_details}

Select one of the below advisors that disapproved the prompt to provide feedback on how to improve it: 
{options}

Think carefully about which aspects of the prompt need improvement and which advisor would be best suited to provide feedback on those aspects.
If you think multiple aspects of the prompt need improvement, select the most suitable advisor to provide feedback on the most critical aspect of the prompt.
"""
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            chain = (
                prompt
                | self.llm.bind_functions(functions=[function_def], function_call="route")
                | JsonOutputFunctionsParser()
            )
            result = chain.invoke({"messages": state})
            print(result["next"])
            return result["next"]

    def update_prompt(self, state: Sequence[BaseMessage]) -> str:
        """
        Updates the prompt with the feedback from the advisor agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to review the conversation above and improve the prompt in light of your core principles.
If you recieve feedback and recommendations for the prompt, respond with a revised version of your previous attempts actioning the feedback.

The success criteria for the prompt are as follows:
{self.criteria}
You will be penalized if the prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Review the conversation carefully as an expert Head AI Engineer.
2. Think carefully about how you can implement the user's feedback.
3. Implement the feedback in a revised version of the prompt.
4. Explcitly go through each success criteria and ensure your revised prompt meets them. If not, repeat from step 2.
5. Explicitly go through each guideline and ensure your changes adhere to them. If not, repeat from step 2.
6. Submit your revised prompt.
"""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text),
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [AIMessage(content=result.content, name="Leader")], "next": self.leader_decision(state)}

    def construct_advisor_graph(self):
        """
        Constructs a graph of advisor agents based on their roles and functions.
        """

        def advisor_node(state, agent):
            return agent.review_prompt(state["messages"], self.criteria)
        
        def leader_node(state):
            return self.update_prompt(state["messages"]) 
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        workflow = StateGraph(AgentState)
        for advisor in self.advisors:
            # Create a node for each advisor agent
            node = functools.partial(advisor_node, agent=advisor)
            workflow.add_node(advisor.position, node)
        workflow.add_node("Leader", leader_node)

        members = [advisor.position for advisor in self.advisors]
        for member in members:
            # We want our advisors to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "Leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("Leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("Leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of advisor agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=self.base_prompt, name="User")],
        }

        # Construct the graph
        graph = self.construct_advisor_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": n},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                if len(s["messages"]) > 1:
                    s["messages"][-1].pretty_print()
                continue
        
        # if not os.path.exists("prompt_history_authoritarian.json"):
        #     with open("prompt_history_authoritarian.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_authoritarian.json", "r") as f:
        #     data = json.load(f)
        #     data.append(s)
            
        # with open("prompt_history_authoritarian.json", "w") as f:
        #     json.dump(data, f, indent=4)

        return s


In [20]:
# llm = ChatAnthropic(temperature=1.0, model="claude-3-haiku-20240307")
llm = ChatOpenAI(temperature=1.0, model="gpt-4o")

In [21]:
# Prompt design team members
style_and_structure_principles = CorePrinciples([
    "Always structure prompts logically for the task",
    "Always use a style and tone in prompts that is appropriate for the task",
    "Always design prompts appropriately for the task's complexity",
])
style_and_structure_expert = AdvisorAgent("Style_and_Structure_Expert", style_and_structure_principles, llm)

conciseness_and_clarity_principles = CorePrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language in prompts",
    "Always avoid ambiguity in prompts",
])
conciseness_and_clarity_expert = AdvisorAgent("Conciseness_and_Clarity_Expert", conciseness_and_clarity_principles, llm)

contextual_relevance_principles = CorePrinciples([
    "Always provide context to help the model understand the task",
    "Always write prompts informed by the context of the task",
    "Always design contextually relevant personas and roles in prompts",
])
contextual_relevance_expert = AdvisorAgent("Contextual_Relevance_Expert", contextual_relevance_principles, llm)

task_alignment_principles = CorePrinciples([
    "Always write prompts that align with the task criteria",
    "Always tailor instructions to the task to guide the model",
    "Always make the task abundantly clear to the model in the prompt"
])
task_alignment_expert = AdvisorAgent("Task_Alignment_Expert", task_alignment_principles, llm)

example_demonstration_principal = CorePrinciples([
    "Always provide examples to help the model understand the task",
    "Always provide examples that cover a range of complexities",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = AdvisorAgent("Example_Demonstration_Expert", example_demonstration_principal, llm)

avoiding_bias_principles = CorePrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = AdvisorAgent("Avoiding_Bias_Expert", avoiding_bias_principles)

incremental_prompting_principles = CorePrinciples([
    "Always write clear step-by-step instructions to guide the model",
    "Always write instructions tailored to an AI model's learning process",
    "Always write instructions appropriate for the task complexity",
])
incremental_prompting_expert = AdvisorAgent("Incremental_Prompting_Expert", incremental_prompting_principles, llm)

programming_logic_principles = CorePrinciples([
    "Always write pormpts following programming logic principles (loops, conditionals, functions, pseudo-code, etc.)",
    "Always structure prompts as if you were writing code",
    "Always leave comments in plain english to explain programming logic",
])
programming_logic_expert = AdvisorAgent("Programming_Logic_Expert", programming_logic_principles, llm)

In [22]:
# Domain team members
mathematics_principles = CorePrinciples([
    "Always stop to think and develop a mathetcically sound plan to solve problems",
    "Always make an initial estimate of the answer before solving the problem",
    "Always use mathetical operators (addition, subtraction, multiplication, division) correctly",
    "Always double-check mathematical calculations",
])
mathematician = AdvisorAgent("Mathematician", mathematics_principles, llm)

word_problem_solving_principles = CorePrinciples([
    "Always read the problem slowly and carefully to identify what the problem is asking you to find",
    "Always list the given facts and unknown facts",
    "Always rewrite the problem and facts in a more organized manner",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = AdvisorAgent("Word_Problem_Solver", word_problem_solving_principles, llm)


In [23]:
base_prompt = "{content}\nPlease output your answer at the end as ##<your answer (among A through C)>."
criteria = """- The prompt MUST instruct the LLM to solve the object tracking problem.
- The prompt MUST include the content placeholder (this is where the object tracking problem will be).
- The prompt MUST instruct the LLM to provide the answer at the end of the output exactly as ##<answer (among A through C)>.
- The prompt MUST instruct the LLM to provide the answer with no spaces between the ## and the answer."""

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
    advisors=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
        # mathematician,
        word_problem_solver,
    ],
    llm = llm
)
for advisor in leader_agent.advisors:
    print("Position: ", advisor.position + "\nCore Principles:\n", advisor.core_principles)


Position:  Style_and_Structure_Expert
Core Principles:
 - Always structure prompts logically for the task
- Always use a style and tone in prompts that is appropriate for the task
- Always design prompts appropriately for the task's complexity
Position:  Conciseness_and_Clarity_Expert
Core Principles:
 - Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always avoid ambiguity in prompts
Position:  Contextual_Relevance_Expert
Core Principles:
 - Always provide context to help the model understand the task
- Always write prompts informed by the context of the task
- Always design contextually relevant personas and roles in prompts
Position:  Task_Alignment_Expert
Core Principles:
 - Always write prompts that align with the task criteria
- Always tailor instructions to the task to guide the model
- Always make the task abundantly clear to the model in the prompt
Position:  Example_Demonstration_Expert
Core Principles:
 - Always provide examples to

In [24]:
result = leader_agent.optimise_prompt()

Approval results: [{'advisor': 'Example_Demonstration_Expert', 'decision': 'True'}, {'advisor': 'Task_Alignment_Expert', 'decision': 'True'}, {'advisor': 'Conciseness_and_Clarity_Expert', 'decision': 'True'}, {'advisor': 'Contextual_Relevance_Expert', 'decision': 'False'}, {'advisor': 'Word_Problem_Solver', 'decision': 'True'}, {'advisor': 'Style_and_Structure_Expert', 'decision': 'True'}, {'advisor': 'Incremental_Prompting_Expert', 'decision': 'True'}]
Task_Alignment_Expert
================================ Human Message =================================
Name: Task_Alignment_Expert

1. Success Criteria Check:
   The current prompt does not meet all the success criteria:
   - It does not explicitly instruct the LLM to solve the object tracking problem.

2. Guidelines Check:
   - It adheres to the guidelines by keeping the placeholder and required formatting instructions intact.

3. Creative Recommendations for Improvement:
   - Clearly instruct the LLM to solve the object tracking probl

In [25]:
result['messages'][-1].pretty_print()

================================== Ai Message ==================================
Name: Leader

**Reviewing the Conversation:**

Based on the feedback from the experts and adhering to the success criteria and guidelines, here is a revised version of the prompt.

**Revised Prompt:**

```plaintext
Please solve the following object tracking problem and provide the answer at the end exactly as ##<your answer (among A through C)> with no spaces between the ## and your answer. {content}
```

**Check Against Success Criteria:**

1. **Instructs the LLM to solve the object tracking problem:**
   - Yes, it says, "Please solve the following object tracking problem..."
2. **Includes the content placeholder:**
   - Yes, it retains `{content}`.
3. **Instructs the LLM to provide the answer in the required format:**
   - Yes, it instructs to "provide the answer at the end exactly as ##<your answer (among A through C)>..."
4. **Instructs LLM to provide the answer with no spaces between ## and the answer

In [13]:
# messages = [
#     ("system", self.system_message),
#     MessagesPlaceholder(variable_name="messages"),
#     ("system", prompt_text),
# ]

# for message in messages:
#     if isinstance(message, tuple):
#         print(f"Message Type: {message[0]}, Content: {message[1]}")
#     else:
#         print(f"Message Type: {type(message).__name__}, Content: {message}")


# def test_message_conversion(messages, template_format="f-string"):
#     for message in messages:
#         try:
#             converted_message = _convert_to_message(message, template_format)
#             print(f"Successfully converted message: {message}")
#         except ValueError as e:
#             print(f"Error converting message: {message}")
#             print(f"ValueError: {e}")

# test_message_conversion(messages)

# try:
#     prompt = ChatPromptTemplate.from_messages(messages, template_format="f-string")
# except ValueError as e:
#     print(f"Error creating ChatPromptTemplate: {e}")
#     print("Messages passed to ChatPromptTemplate:")
#     for message in messages:
#         if isinstance(message, tuple):
#             print(f"Message Type: {message[0]}, Content: {message[1]}")
#         else:
#             print(f"Message Type: {type(message).__name__}, Content: {message}")
#     raise

### Concurrent Runs

In [ ]:
base_prompt = "Classify the grammar in the text as correct or incorrect: {content}"
criteria = "The prompt should instruct the LLM to output either 'correct' or 'incorrect' based on the grammar in the text."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_1.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_2.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_3.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

In [ ]:
# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_advisors=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next advisor to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

In [ ]:
print(final_result)